# python play

In [1]:
for x in [1,2,3,4,5]: # transactions
    for y in "abcde": # txins
        if y == "b":
            break
        print(x, y)

1 a
2 a
3 a
4 a
5 a


# trying out signatures

In [2]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from Crypto.Signature import DSS

def make_keypair():
    privkey = ECC.generate(curve='P-256')
    pubkey = privkey.public_key()
    return privkey, pubkey 

def make_key(hex):
    key = ECC.import_key(bytes.fromhex(hex))
    return key

def make_hex_key(key):
    binary = key.public_key().export_key(format='DER')
    return binary.hex()

def sign_something(priv_key, something):
    signer = DSS.new(priv_key, 'fips-186-3')
    h = SHA256.new(something)
    signature = signer.sign(h)
    return signature.hex()

def verify_signature(pub_key, something, signature):
    h = SHA256.new(something)
    verifier = DSS.new(pub_key, 'fips-186-3')
    try:
        verifier.verify(h, signature)
        print ("The message is authentic.")
    except ValueError:
        print ("The message is not authentic.")

priv_key, pub_key = make_keypair()
priv_key2, pub_key2 = make_keypair()

# print(pub_key)

message = b'I accept this'
signature = sign_something(priv_key, message)
bitas = bytes.fromhex(signature)
verify_signature(pub_key, message, bitas)

hexkey = make_hex_key(pub_key)

make_key(hexkey)

priv_key.public_key()

The message is authentic.


EccKey(curve='NIST P-256', point_x=84909780357244496161981393799060568445037180437850959173362728121404803006270, point_y=49979488808253775045080323421559923784601969226309406488508769902697307619270)

# trying out transactions

In [3]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"
bob_privkey, bob_pubkey = make_keypair()

# f = open('privatekey.der', 'wt')
# f.write(bob_privkey.export_key(format='DER').hex())
# f.close()

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

unsignedTxIns = bobs_gift_transaction.txins
for uTxIn in unsignedTxIns:
    signature = sign_something(bob_privkey, bobs_gift_transaction.id.encode())
    uTxIn.signature = signature

# Verifying that thing is signed
signa = bytes.fromhex(unsignedTxIns[0].signature)
verify_signature(bob_pubkey, bobs_gift_transaction.id.encode(), signa)

The message is authentic.


# full-node toy model 

In [5]:
from blockchain import *
from transaction import *
from wallet import *
from transaction_pool import *

# maak een wallet aan
wallet_name = 'toufiks-wallet'
init_wallet(wallet_name)
priv_key = get_key_from_wallet(wallet_name)

# maak de blockchain aan
blockchain = Blockchain()



wallet already exists


EccKey(curve='NIST P-256', point_x=78751158075622722828768602150341177141674699718105021827095074887314665369272, point_y=17863991188006476163937486212084310646861704467451798368889507225270856066180, d=104774712188689678462332036671138106001982490608801461724002014348531904996599)